In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
# !pip install --upgrade pandas_profiling
!pip install --upgrade category_encoders
# !pip install --upgrade sweetviz

In [42]:
import numpy as np
import pandas as pd
import json
import category_encoders as ce
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [43]:
import sklearn
import lightgbm

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import ElasticNet
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from xgboost.sklearn import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor

from sklearn.preprocessing import StandardScaler

#Modeling

##Load dataset

In [44]:
final_cars = pd.read_csv('/content/drive/MyDrive/Colab_workspaces/Python Playground/New folder/final_cars.csv')
cars_org = final_cars.copy()

In [45]:
final_cars.sort_values(by='km_driven', ascending=False).head(30)

,brand,engine_capacity,fuels,km_driven,model,origin,price,seats,transmission,type,...,gasoline,hybrid,imported,coupe,crossover,hatchback,pickup,sedan,suv,van
9018,toyota,1.30,gasoline,4294967295,toyota yaris,imported,235000000,5,automatic,hatchback,...,1,0,1,0,0,1,0,0,0,0
8293,toyota,1.50,gasoline,210000000,toyota vios,domestic,484000000,4,automatic,sedan,...,1,0,0,0,0,0,0,1,0,0
3918,kia,1.10,gasoline,152000000,kia morning,domestic,145000000,5,manual,hatchback,...,1,0,0,0,0,1,0,0,0,0
1193,ford,2.40,diesel,120000000,ford transit,domestic,395000000,4,manual,van,...,0,0,0,0,0,0,0,0,0,1
3670,kia,2.20,diesel,70000000,kia sedona,domestic,920000000,5,automatic,van,...,0,0,0,0,0,0,0,0,0,1
3543,kia,1.60,gasoline,70000000,kia k3,domestic,412000000,4,automatic,sedan,...,1,0,0,0,0,0,0,1,0,0
8762,toyota,2.00,gasoline,42500000,toyota innova,domestic,230000000,5,manual,crossover,...,1,0,0,0,1,0,0,0,0,0
6141,mercedes,2.20,diesel,30546645,mercedes sprinter,domestic,190000000,4,manual,van,...,0,0,0,0,0,0,0,0,0,1
444,chevrolet,1.20,gasoline,20000000,chevrolet spark,domestic,170000000,5,manual,van,...,1,0,0,0,0,0,0,0,0,1
1480,ford,2.40,diesel,16000000,ford transit,domestic,330000000,4,manual,van,...,0,0,0,0,0,0,0,0,0,1


##Remove outliers

In [46]:
# final_cars.drop(index=[32267, 43234], inplace=True) #Contains wrong information

In [47]:
final_cars.drop(index=final_cars[final_cars['seats'] == 'Khác'].index, inplace=True)
final_cars['seats'] = final_cars.seats.astype(float)

In [48]:
final_cars = final_cars[(np.abs(stats.zscore(final_cars.price)) < 2.8)]

In [49]:
replace_cc = final_cars[final_cars.engine_capacity == 2000].index
final_cars.engine_capacity.loc[replace_cc] = 2.0

In [50]:
final_cars = final_cars[final_cars.seats<30]
final_cars = final_cars[final_cars.year>2007]
final_cars = final_cars[final_cars.km_driven < 1e+08]
final_cars = final_cars[final_cars.km_driven > 0]

In [51]:
index_drop = []
list_drop = [NameError]
for i in final_cars.model.unique():
  idx = final_cars[final_cars.model == i][(np.abs(stats.zscore(final_cars[final_cars.model == i].price)) > 2.75)].index.tolist()
  index_drop.append(idx)

for i in index_drop:
  for k in i:
    list_drop.append(k)

list_drop = list_drop[1:]
len(list_drop)

44

In [52]:
final_cars.drop(index = list_drop, inplace=True)

In [53]:
final_cars.sort_values(by='price', ascending = False)

,brand,engine_capacity,fuels,km_driven,model,origin,price,seats,transmission,type,...,gasoline,hybrid,imported,coupe,crossover,hatchback,pickup,sedan,suv,van
5389,mercedes,2.0,gasoline,12000,mercedes glc,domestic,2290000000,5.0,automatic,suv,...,1,0,0,0,0,0,0,0,1,0
5181,mercedes,2.0,gasoline,17000,mercedes glc,domestic,2290000000,5.0,automatic,suv,...,1,0,0,0,0,0,0,0,1,0
4391,lexus,4.6,gasoline,62000,lexus gx,imported,2290000000,5.0,automatic,suv,...,1,0,1,0,0,0,0,0,1,0
7891,toyota,2.7,gasoline,16000,toyota prado,imported,2289000000,5.0,automatic,suv,...,1,0,1,0,0,0,0,0,1,0
5502,mercedes,2.0,gasoline,17000,mercedes glc,domestic,2289000000,5.0,automatic,suv,...,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2518,hyundai,1.1,gasoline,100000,hyundai getz,imported,125000000,5.0,manual,hatchback,...,1,0,1,0,0,1,0,0,0,0
3596,kia,1.1,gasoline,12000,kia morning,domestic,125000000,5.0,manual,hatchback,...,1,0,0,0,0,1,0,0,0,0
3368,kia,1.1,gasoline,184000,kia morning,domestic,123000000,5.0,manual,hatchback,...,1,0,0,0,0,1,0,0,0,0
468,chevrolet,0.8,gasoline,128000,chevrolet spark,domestic,122000000,5.0,manual,hatchback,...,1,0,0,0,0,1,0,0,0,0


In [54]:
final_cars

,brand,engine_capacity,fuels,km_driven,model,origin,price,seats,transmission,type,...,gasoline,hybrid,imported,coupe,crossover,hatchback,pickup,sedan,suv,van
1,bmw,2.0,gasoline,57000,bmw 3,imported,929000000,4.0,automatic,sedan,...,1,0,1,0,0,0,0,1,0,0
2,bmw,2.0,gasoline,60000,bmw 5,imported,1255000000,4.0,automatic,sedan,...,1,0,1,0,0,0,0,1,0,0
3,bmw,2.0,gasoline,28500,bmw 3,imported,1199000000,4.0,automatic,sedan,...,1,0,1,0,0,0,0,1,0,0
4,bmw,2.0,gasoline,45000,bmw 3,imported,920000000,4.0,automatic,sedan,...,1,0,1,0,0,0,0,1,0,0
5,bmw,1.6,gasoline,40000,bmw 5,imported,1850000000,4.0,automatic,sedan,...,1,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9289,vinfast,2.0,gasoline,4800,vinfast lux a 2.0,domestic,815000000,4.0,automatic,sedan,...,1,0,0,0,0,0,0,1,0,0
9290,vinfast,1.4,gasoline,4200,vinfast fadil,domestic,342000000,5.0,automatic,hatchback,...,1,0,0,0,0,1,0,0,0,0
9291,vinfast,2.0,gasoline,9000,vinfast lux sa 2.0,domestic,1230000000,5.0,automatic,suv,...,1,0,0,0,0,0,0,0,1,0
9292,vinfast,2.0,gasoline,15000,vinfast lux sa 2.0,domestic,1170000000,5.0,automatic,suv,...,1,0,0,0,0,0,0,0,1,0


In [57]:
final_cars.drop(['type', 'fuels', 'transmission', 'origin'], axis=1, inplace= True)


In [58]:
final_cars

,brand,engine_capacity,km_driven,model,price,seats,year,manual,gasoline,hybrid,imported,coupe,crossover,hatchback,pickup,sedan,suv,van
1,bmw,2.0,57000,bmw 3,929000000,4.0,2016,0,1,0,1,0,0,0,0,1,0,0
2,bmw,2.0,60000,bmw 5,1255000000,4.0,2016,0,1,0,1,0,0,0,0,1,0,0
3,bmw,2.0,28500,bmw 3,1199000000,4.0,2016,0,1,0,1,0,0,0,0,1,0,0
4,bmw,2.0,45000,bmw 3,920000000,4.0,2016,0,1,0,1,0,0,0,0,1,0,0
5,bmw,1.6,40000,bmw 5,1850000000,4.0,2018,0,1,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9289,vinfast,2.0,4800,vinfast lux a 2.0,815000000,4.0,2019,0,1,0,0,0,0,0,0,1,0,0
9290,vinfast,1.4,4200,vinfast fadil,342000000,5.0,2021,0,1,0,0,0,0,1,0,0,0,0
9291,vinfast,2.0,9000,vinfast lux sa 2.0,1230000000,5.0,2019,0,1,0,0,0,0,0,0,0,1,0
9292,vinfast,2.0,15000,vinfast lux sa 2.0,1170000000,5.0,2020,0,1,0,0,0,0,0,0,0,1,0


##Encoding categorical

In [59]:
final_cars['brand'] = ce.JamesSteinEncoder().fit_transform(final_cars['brand'], final_cars['price'])
final_cars['model'] = ce.JamesSteinEncoder().fit_transform(final_cars['model'], final_cars['price'])

##Train test split

In [60]:
X = final_cars.drop(["price"],axis=1)
y = final_cars['price']

In [61]:
VERBOSE = 0
SEED = 42

In [72]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

X_train , X_test, y_train, y_test = train_test_split(X, y, random_state = SEED , test_size=0.2)

# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

In [63]:
def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

In [64]:
regressors = [
    LinearRegression(),
    # SGDRegressor(),
    # ElasticNet(),
    # KernelRidge(),
    # SVR(),
    XGBRegressor(),
    RandomForestRegressor(),
    ExtraTreesRegressor(), 
    GradientBoostingRegressor()
]

In [65]:
res = pd.DataFrame(columns=['Model', 'RMSE', 'R-Squared'])
RMSE = []
R2 = []
for num, regressor in enumerate(regressors):
    reg = regressor.fit(X_train, y_train)
    pred = reg.predict(X_test)
    rmse = np.sqrt(mean_squared_error(pred, y_test))
    r2 = r2_score(pred, y_test)
    RMSE.append(str(np.around(rmse)))
    R2.append(str(round(r2, 2)*100) + '%')
res['Model'] = ['LinearRegression', 'XGBRegressor', 'RandomForestRegressor', 'ExtraTreesRegressor', 'GradientBoostingRegressor']
res['RMSE'] = RMSE
res['R-Squared'] = R2
res

[11:07:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


,Model,RMSE,R-Squared
0,LinearRegression,176729934.0,78.0%
1,XGBRegressor,90998270.0,95.0%
2,RandomForestRegressor,65470868.0,97.0%
3,ExtraTreesRegressor,70372704.0,97.0%
4,GradientBoostingRegressor,88976663.0,95.0%


###GridSearchCV

In [66]:
# from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

# param_grid2 = {
#     "learning_rate": [0.2, 0.25],
#     "n_estimators": [i for i in range(270,300,10)],
#     'max_depth': [i for i in range(13,19)],
#     "num_leaves": [i for i in range(30,50)],
#     'min_data_in_leaf': [20,23,24,19],
#     'reg_alpha': [0.1, 0.2],
#     'reg_lambda': [0.02,0.01]
# }

# regressor = LGBMRegressor()
# gscv_lgbm2 = GridSearchCV(regressor, param_grid2, scoring='neg_mean_squared_error', cv=3)
# gscv_lgbm2.fit(X_train, y_train)
# yhat_lgbmr = gscv_lgbm2.predict(X_test)
# print(gscv_lgbm2.best_params_)
# np.sqrt(mean_squared_error(y_test, yhat_lgbmr))

# # for x in range(0,50):
# #   gscv_lgbm2 = RandomizedSearchCV(regressor, param_grid2, scoring='neg_mean_squared_error', cv=3)
# #   gscv_lgbm2.fit(X_train, y_train)
# #   yhat_lgbmr = gscv_lgbm2.predict(X_test)
# #   k = np.sqrt(mean_squared_error(y_test, yhat_lgbmr))
# #   if k < 67596171:
# #     print(gscv_lgbm2.best_params_)
# #     print(k)

66.0 : {'reg_lambda': 0.01, 'reg_alpha': 0.2, 'num_leaves': 40, 'n_estimators': 300, 'min_data_in_leaf': 19, 'max_depth': 13, 'learning_rate': 0.2}

66.1 : {'reg_lambda': 0.01, 'reg_alpha': 0.1, 'num_leaves': 47, 'n_estimators': 310, 'min_data_in_leaf': 20, 'max_depth': 18, 'learning_rate': 0.2}

65.4 : {'reg_lambda': 0.01, 'reg_alpha': 0.4, 'num_leaves': 39, 'n_estimators': 245, 'min_data_in_leaf': 20, 'max_depth': 13, 'learning_rate': 0.25}

65.2: {'reg_lambda': 0.02, 'reg_alpha': 0.2, 'num_leaves': 48, 'n_estimators': 280, 'min_data_in_leaf': 19, 'max_depth': 14, 'learning_rate': 0.2}

In [67]:
regressor = LGBMRegressor()
params = {'reg_lambda': 0.02, 'reg_alpha': 0.2, 'num_leaves': 48, 'n_estimators': 280, 'min_data_in_leaf': 19, 'max_depth': 14, 'learning_rate': 0.2}
# params = {'reg_lambda': 0.01, 'reg_alpha': 0.1, 'num_leaves': 47, 'n_estimators': 310, 'min_data_in_leaf': 20, 'max_depth': 18, 'learning_rate': 0.2}
regressor.set_params(**params)
reg = regressor.fit(X_train, y_train)
pred = reg.predict(X_test)
pred = np.around(pred, decimals=-6) #lam tron gia tien
rmse = np.sqrt(mean_squared_error(pred, y_test))
print(f'{regressor} has rmse of {rmse}')

LGBMRegressor(learning_rate=0.2, max_depth=14, min_data_in_leaf=19,
              n_estimators=280, num_leaves=48, reg_alpha=0.2, reg_lambda=0.02) has rmse of 65485959.36870332


In [68]:
from sklearn.metrics import r2_score

r2_score(pred, y_test)

0.9742750964168384

##Feature Importances plot

In [69]:
def plotImp(model, X , num = 20, fig_size = (40, 20)):
    feature_imp = pd.DataFrame({'Value':model.feature_importances_,'Feature':X.columns})
    plt.figure(figsize=fig_size)
    sns.set(font_scale = 5)
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", 
                                                        ascending=False)[0:num])
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances-01.png')
    plt.show()

In [73]:
import plotly.express as px

feature_imp = pd.DataFrame({'Value':reg.feature_importances_,'Feature':X_train.columns})
dat = feature_imp.sort_values(by='Value',ascending=True)
fig = px.bar(dat, x="Value", y="Feature", title="Feature Importances")
fig.show()